<a href="https://colab.research.google.com/github/GawainGan/LLM/blob/main/LangGraph/Lesson_1_Simple_ReAct_Agent_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 1: Simple ReAct Agent from Scratch

In [ ]:
# based on https://til.simonwillison.net/llms/python-react-pattern

In [ ]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

In [ ]:
client = OpenAI()

In [ ]:
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello world"}]
)

In [ ]:
chat_completion.choices[0].message.content

'Hello! How can I assist you today?'

In [ ]:
class Agent:
    def __init__(self, system=""):
      '接收一个可选的系统消息并初始化消息列表'
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
      '使得 Agent 实例可以像函数一样被调用，接收一个消息，执行聊天逻辑，并返回回复'
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
      '辅助函数，用于实际调用 OpenAI API 并获取模型的回复'
        completion = client.chat.completions.create(
                        model="gpt-4o",
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content


初始化提示和定义函数：
- prompt：提供了一个说明性的多行字符串，解释了聊天机器人的工作方式，包括思考（Thought）、行动（Action）、暂停（PAUSE）和观察（Observation）的循环过程。
- calculate 函数：执行一个数学计算并返回结果。
- average_dog_weight 函数：根据给定的狗的品种返回平均体重。该函数支持三种狗的品种，并对未知品种给出默认的平均体重。

In [ ]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [ ]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier":
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

## Agent 1

### Prompt 1

In [ ]:
abot = Agent(prompt) # 创建了一个 Agent 类的实例，使用先前定义的 prompt 作为初始化输入

In [ ]:
result = abot("How much does a toy poodle weigh?") # 调用 abot 实例，并向其发送关于玩具贵宾犬平均体重的查询
print(result) # 在第一个查询后，输出了机器人的“思考”和计划采取的“行动”，然后暂停

Thought: I should look up the average weight of a Toy Poodle using the average_dog_weight action.
Action: average_dog_weight: Toy Poodle
PAUSE


In [ ]:
result = average_dog_weight("Toy Poodle") # 直接调用 average_dog_weight 函数以获取玩具贵宾犬的平均体重，并将结果存储在 result 中

In [ ]:
result

'a toy poodles average weight is 7 lbs'

### Prompt 2

In [ ]:
next_prompt = "Observation: {}".format(result) # 生成一个观察阶段的提示，该prompt要包括上一步操作得到的结果

In [ ]:
abot(next_prompt) # 将新的prompt放入创建的Agent中，Agent根据观察结果生成一个答案。这是对用户最初查询的直接相应

'Answer: A Toy Poodle weighs an average of 7 lbs.'

上面直接生成的result就是Observation的内容，其根据是在Agent初始化的Prompt里就有这个要求：

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs

In [ ]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

在上面创建Agent的 __init__ & __call__ 中，self.message是会自动存储在array中的：

    class Agent:
        def __init__(self, system=""):
          '接收一个可选的系统消息并初始化消息列表'
            self.system = system
            self.messages = []
            if self.system:
                self.messages.append({"role": "system", "content": system})

        def __call__(self, message):
          '使得 Agent 实例可以像函数一样被调用，接收一个消息，执行聊天逻辑，并返回回复'
            self.messages.append({"role": "user", "content": message})
            result = self.execute()
            self.messages.append({"role": "assistant", "content": result})
            return result

## Agent 2

### Prompt 1

In [ ]:
abot = Agent(prompt)

In [ ]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

'Thought: I need to find the average weight of both a Border Collie and a Scottish Terrier, then sum these weights to get the combined weight of the two dogs.\nAction: average_dog_weight: Border Collie\nPAUSE'

In [ ]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [ ]:
abot(next_prompt)

'Thought: Now I need to find the average weight of a Scottish Terrier.\nAction: average_dog_weight: Scottish Terrier\nPAUSE'

In [ ]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [ ]:
abot(next_prompt)

'Thought: I now have the average weights of both dogs. I will sum these weights to get the combined weight.\nAction: calculate: 37 + 20\nPAUSE'

In [ ]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [ ]:
abot(next_prompt)

'Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.'

### Add loop

In [ ]:
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action
# 它匹配以 “Action:” 开头的字符串，并提取行动名称和输入参数

In [ ]:
def query(question, max_turns=5):
  """
  接收一个问题和最大交互次数（max_turns），用来控制交互的次数
  使用 Agent 实例进行交互，并根据返回的结果确定下一步行动。通过循环追踪对话过程，每次循环处理一个行动和对应的观察结果

  交互循环：
	•	在每个循环中，首先调用 Agent 实例处理当前的提示（next_prompt），然后打印结果。
	•	使用正则表达式从结果中查找行动指令。如果找到了行动指令，它会执行对应的已知行动。
	•	如果识别的行动在 known_actions 字典中，则执行该行动并产生观察结果，更新 next_prompt 以继续交互
  """
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [ ]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I need to find the average weight of both a Border Collie and a Scottish Terrier, then sum these weights to get the combined weight.
Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: Now I need to find the average weight of a Scottish Terrier.
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: I now have the average weights of both dogs. I will sum these weights to get the combined weight.
Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.


### 步骤 1: 初始化和问题输入
- **Prompt 的内容** 定义了 `Agent` 类实例如何响应问题。这个初始化阶段包括加载和设置 `Agent` 的环境，如加载聊天模型和理解任务的指导提示。
- **用户问题的输入**：用户提出的问题被视为触发整个聊天处理流程的起点。例如，用户问：“我有两只狗，边境牧羊犬和苏格兰梗，它们的总体重是多少？”

### 步骤 2: 理解问题并思考回答策略
- **Thought**：`Agent` 首先要理解问题的含义，并决定如何回答。它会输出一个以 "Thought:" 开头的描述，表达对问题的理解和打算采取的初步策略。例如，它可能认为需要分别获取两种狗的平均体重。

### 步骤 3: 执行行动和暂停
- **Action**：基于前一步的思考，`Agent` 决定执行一个或多个行动来获取所需信息。这些行动对应于预定义的函数，如 `calculate` 和 `average_dog_weight`。
- **PAUSE**：行动执行后，脚本将返回一个 "PAUSE"，这是一个等待状态，表示已完成当前行动，等待进一步处理或下一个行动。

### 步骤 4: 观察结果
- **Observation**：行动执行后的结果被视为观察，这些结果是对之前行动的直接回应。`Agent` 将这些观察结果用于进一步的处理或作为最终答案的一部分。

### 步骤 5: 分析观察并决定是否需要额外行动
- 如果单次行动未能解决问题，`Agent` 可能需要根据当前的观察结果再次执行其他行动。这可能涉及进一步的计算或信息获取。

### 步骤 6: 输出答案
- **Answer**：一旦所有必要的信息被收集和计算，例如两种狗的体重总和，`Agent` 将格式化并输出一个答案，通常以 "Answer:" 开头，明确回答用户的初始问题。

### 步骤 7: 循环和交互管理
- 交互过程可能需要通过多个 "Thought-Action-PAUSE-Observation" 循环来处理，每个循环都是围绕用户问题的解决策略构建的。在给定的代码中，这通过在 `query` 函数中的 `while` 循环实现，该循环控制整个交互过程。

通过这种结构化的多步骤处理流程，聊天机器人可以有效地处理复杂的查询，从而提供连贯、逻辑性强的用户交互体验。这种方法不仅保证了问题能够得到准确的解答，也增强了系统的可扩展性和维护性。